In [1]:
import torch
import numpy as np
import pickle

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Product, ConstantKernel as C

import gym_sin
from gym import spaces

from utilities.arguments import get_args
from learner.posterior_multi_task import PosteriorMTAgent
from inference.inference_network import InferenceNetwork
from task.ExploreTaskGenerator import ExploreTaskGenerator
from utilities.folder_management import handle_folder_creation


In [2]:
env_name = "gaussexplore-v0"

action_space = spaces.Box(low=np.array([-1]), high=np.array([1]))

latent_dim = 1

device = "cpu"

x_min=-100
x_max=100
noise_std=0.001
std=10
mean_max=60 
mean_min=40

vae_min_seq = 1
vae_max_seq = 20

max_old = [100, 25]
min_old = [-100, 0]

obs_shape = (2,)

In [3]:
task_generator = ExploreTaskGenerator(x_min=x_min, x_max=x_max, noise_std=noise_std, std=std, mean_max=mean_max,
                                     mean_min=mean_min)
f = task_generator.create_task_family(n_tasks=5000, n_batches=1, test_perc=0, batch_size=1)

In [4]:
task_generator.sample_pair_tasks(1)

([{'min_x': -100,
   'max_x': 100,
   'noise_std': 0.001,
   'std': 10,
   'mean': 99.79561614990234,
   'scale_reward': False,
   'amplitude': 1}],
 None,
 [tensor([[50.],
          [20.]])],
 tensor([[99.7956]]))

In [5]:
vi = InferenceNetwork(n_in=4, z_dim=latent_dim)
vi_optim = torch.optim.Adam(vi.parameters(), lr=1e-3)

In [7]:
agent = PosteriorMTAgent(action_space=action_space, device=device, gamma=1,
                                 num_steps=20, num_processes=32,
                                 clip_param=0.1, ppo_epoch=4,
                                 num_mini_batch=8,
                                 value_loss_coef=0.5,
                                 entropy_coef=0.001,
                                 lr=0.00001,
                                 eps=1e-6, max_grad_norm=0.5,
                                 use_linear_lr_decay=False,
                                 use_gae=False,
                                 gae_lambda=0.95,
                                 use_proper_time_limits=False,
                                 obs_shape=obs_shape,
                                 latent_dim=latent_dim,
                                 recurrent_policy=False,
                                 hidden_size=8,
                                 use_elu=True,
                                 variational_model=vi,
                                 vae_optim=vi_optim,
                                 rescale_obs=True,
                                 max_old=max_old,
                                 min_old=min_old,
                                 vae_min_seq=vae_min_seq,
                                 vae_max_seq=vae_max_seq,
                                 max_action=x_max, 
                                 min_action=x_min,
                        use_time=False, rescale_time=None, max_time=None,
                        max_sigma=30,
                        use_decay_kld=True,
                        decay_kld_rate=1)

In [ ]:
res_eval, res_vae, test_list = agent.train(training_iter=60000,
                                           env_name=env_name,
                                           seed=0,
                                           task_generator=task_generator,
                                           eval_interval=100,
                                           log_dir=".",
                                           use_env_obs=False,
                                           init_vae_steps=1,
                                           sw_size=10,
                                           num_random_task_to_eval=32,
                                           num_test_processes=2,
                                           use_data_loader=False,
                                           gp_list_sequences=[],
                                           prior_sequences=[],
                                           init_prior_test_sequences=[],
                                           verbose=True
                                          )

Epoch 0 MSE DIR 4124.00537109375 MSE VAR 0.2598961591720581 KLD 3.428971529006958
Epoch 0 MSE DIR 2632.0927734375 MSE VAR 0.6786916255950928 KLD 10.192231178283691
Epoch 100 MSE DIR 2387.422607421875 MSE VAR 0.06614695489406586 KLD 2.2854981422424316
Epoch 100 / 60000
Evaluation...
Evaluation using 32 tasks. Mean reward: 1.9142990624999998
Epoch 200 MSE DIR 713.0827026367188 MSE VAR 0.11949768662452698 KLD 0.9908828735351562
Epoch 200 / 60000
Evaluation...
Evaluation using 32 tasks. Mean reward: 2.16761584375
Epoch 300 MSE DIR 836.0440063476562 MSE VAR 0.0706048384308815 KLD 0.2587652802467346
Epoch 300 / 60000
Evaluation...
Evaluation using 32 tasks. Mean reward: 2.5071226875
Epoch 400 MSE DIR 915.992431640625 MSE VAR 0.0824493020772934 KLD 0.15358604490756989
Epoch 400 / 60000
Evaluation...
Evaluation using 32 tasks. Mean reward: 2.127427125
Epoch 500 MSE DIR 831.2276000976562 MSE VAR 0.06856021285057068 KLD 0.4749152362346649
Epoch 500 / 60000
Evaluation...
Evaluation using 32 tasks

In [8]:
res_eval, res_vae, test_list = agent.train(training_iter=10000,
                                           env_name=env_name,
                                           seed=0,
                                           task_generator=task_generator,
                                           eval_interval=100,
                                           log_dir=".",
                                           use_env_obs=False,
                                           num_vae_steps=1,
                                           init_vae_steps=1,
                                           sw_size=10,
                                           num_random_task_to_eval=32,
                                           num_test_processes=2,
                                           use_data_loader=False,
                                           gp_list_sequences=[],
                                           prior_sequences=[],
                                           init_prior_test_sequences=[],
                                           verbose=True
                                          )

Epoch 0 MSE DIR 1102.126220703125 MSE VAR 0.5106451511383057 KLD 12.363800048828125
Epoch 0 MSE DIR 61.23255157470703 MSE VAR 0.4919677674770355 KLD 16.346757888793945
Epoch 0 MSE DIR 107.28985595703125 MSE VAR 0.48426467180252075 KLD 16.97114372253418
Epoch 0 MSE DIR 87.6654281616211 MSE VAR 0.47243523597717285 KLD 23.340375900268555
Epoch 0 MSE DIR 73.08710479736328 MSE VAR 0.4773685932159424 KLD 25.774188995361328
Epoch 0 MSE DIR 99.29734802246094 MSE VAR 0.4914470911026001 KLD 22.424747467041016
Epoch 0 MSE DIR 134.47625732421875 MSE VAR 0.5051237344741821 KLD 16.921104431152344
Epoch 0 MSE DIR 45.868141174316406 MSE VAR 0.4941844046115875 KLD 19.985750198364258
Epoch 0 MSE DIR 187.98968505859375 MSE VAR 0.4844704866409302 KLD 17.062307357788086
Epoch 0 MSE DIR 183.72235107421875 MSE VAR 0.4779428243637085 KLD 21.491130828857422
Epoch 0 MSE DIR 187.09422302246094 MSE VAR 0.47421103715896606 KLD 14.130449295043945
Epoch 0 MSE DIR 324.5115966796875 MSE VAR 0.49371346831321716 KLD 17.

Epoch 0 MSE DIR 141.29693603515625 MSE VAR 0.47057539224624634 KLD 23.040828704833984
Epoch 0 MSE DIR 61.44329071044922 MSE VAR 0.4773663878440857 KLD 23.273666381835938
Epoch 0 MSE DIR 230.534912109375 MSE VAR 0.48569005727767944 KLD 11.099896430969238
Epoch 0 MSE DIR 65.67581939697266 MSE VAR 0.5192599296569824 KLD 21.584991455078125
Epoch 100 / 10000
Evaluation...
Evaluation using 32 tasks. Mean reward: 15.163866562499999
Epoch 0 MSE DIR 87.88123321533203 MSE VAR 0.50455242395401 KLD 15.208534240722656
Epoch 0 MSE DIR 40.37436294555664 MSE VAR 0.4782692790031433 KLD 19.356348037719727
Epoch 0 MSE DIR 106.21256256103516 MSE VAR 0.4635176360607147 KLD 21.250804901123047
Epoch 0 MSE DIR 101.51525115966797 MSE VAR 0.4706513583660126 KLD 19.620582580566406
Epoch 0 MSE DIR 68.88744354248047 MSE VAR 0.49311041831970215 KLD 12.761812210083008
Epoch 0 MSE DIR 65.1770248413086 MSE VAR 0.5100916624069214 KLD 16.940345764160156
Epoch 0 MSE DIR 190.8408203125 MSE VAR 0.5099077820777893 KLD 14.67

Epoch 0 MSE DIR 9.867161750793457 MSE VAR 0.48726195096969604 KLD 19.526012420654297
Epoch 0 MSE DIR 31.774131774902344 MSE VAR 0.48194006085395813 KLD 22.180946350097656
Epoch 0 MSE DIR 657.4150390625 MSE VAR 0.48657456040382385 KLD 14.701711654663086
Epoch 0 MSE DIR 45.95131301879883 MSE VAR 0.4983396530151367 KLD 18.866907119750977
Epoch 0 MSE DIR 27.07884979248047 MSE VAR 0.4901244640350342 KLD 21.86675262451172
Epoch 0 MSE DIR 24.114336013793945 MSE VAR 0.4846692383289337 KLD 17.397783279418945
Epoch 0 MSE DIR 48.213321685791016 MSE VAR 0.48692312836647034 KLD 16.805139541625977
Epoch 200 / 10000
Evaluation...
Evaluation using 32 tasks. Mean reward: 14.186022031250001
Epoch 0 MSE DIR 21.0980281829834 MSE VAR 0.4818415641784668 KLD 16.96601104736328
Epoch 0 MSE DIR 71.22007751464844 MSE VAR 0.4911867678165436 KLD 20.355897903442383
Epoch 0 MSE DIR 22.282121658325195 MSE VAR 0.4940316081047058 KLD 19.638656616210938
Epoch 0 MSE DIR 20.045873641967773 MSE VAR 0.4837679862976074 KLD 2

Epoch 0 MSE DIR 62.24097442626953 MSE VAR 0.49273622035980225 KLD 16.94883918762207
Epoch 0 MSE DIR 55.49863815307617 MSE VAR 0.4910373091697693 KLD 17.860841751098633
Epoch 0 MSE DIR 46.79127883911133 MSE VAR 0.48874518275260925 KLD 18.00894546508789
Epoch 0 MSE DIR 71.09456634521484 MSE VAR 0.4870077073574066 KLD 23.721355438232422
Epoch 0 MSE DIR 270.919921875 MSE VAR 0.46819907426834106 KLD 8.20993423461914
Epoch 0 MSE DIR 74.04637908935547 MSE VAR 0.49559450149536133 KLD 22.579376220703125
Epoch 0 MSE DIR 33.47499465942383 MSE VAR 0.497993528842926 KLD 20.056655883789062
Epoch 0 MSE DIR 58.39998245239258 MSE VAR 0.49654385447502136 KLD 20.153209686279297
Epoch 0 MSE DIR 14.626851081848145 MSE VAR 0.48243752121925354 KLD 18.788986206054688
Epoch 0 MSE DIR 89.05166625976562 MSE VAR 0.47537103295326233 KLD 18.51671600341797
Epoch 300 / 10000
Evaluation...
Evaluation using 32 tasks. Mean reward: 15.602823781249999
Epoch 0 MSE DIR 22.99711036682129 MSE VAR 0.48044097423553467 KLD 20.98

Epoch 0 MSE DIR 127.16033172607422 MSE VAR 0.5023034811019897 KLD 18.921062469482422
Epoch 0 MSE DIR 57.850711822509766 MSE VAR 0.4871797263622284 KLD 15.997712135314941
Epoch 0 MSE DIR 65.6239242553711 MSE VAR 0.4752005934715271 KLD 20.420581817626953
Epoch 0 MSE DIR 153.79180908203125 MSE VAR 0.48138466477394104 KLD 17.66363525390625
Epoch 0 MSE DIR 83.01224517822266 MSE VAR 0.49148058891296387 KLD 19.094173431396484
Epoch 0 MSE DIR 83.08248138427734 MSE VAR 0.4932485520839691 KLD 13.506078720092773
Epoch 0 MSE DIR 50.66714859008789 MSE VAR 0.4967525601387024 KLD 18.89590835571289
Epoch 0 MSE DIR 289.33349609375 MSE VAR 0.48430532217025757 KLD 15.248037338256836
Epoch 0 MSE DIR 17.967926025390625 MSE VAR 0.47943076491355896 KLD 14.554520606994629
Epoch 0 MSE DIR 180.1589813232422 MSE VAR 0.48723888397216797 KLD 18.810781478881836
Epoch 0 MSE DIR 74.10955810546875 MSE VAR 0.49077096581459045 KLD 20.516202926635742
Epoch 0 MSE DIR 82.7069320678711 MSE VAR 0.49163979291915894 KLD 21.748

...

Epoch 0 MSE DIR 227.3913116455078 MSE VAR 0.49744540452957153 KLD 15.215030670166016
Epoch 0 MSE DIR 52.38847732543945 MSE VAR 0.49686238169670105 KLD 22.532123565673828
Epoch 0 MSE DIR 31.53994369506836 MSE VAR 0.48692217469215393 KLD 19.397043228149414
Epoch 0 MSE DIR 361.0370178222656 MSE VAR 0.4743519425392151 KLD 20.8346004486084
Epoch 0 MSE DIR 296.81085205078125 MSE VAR 0.4850369989871979 KLD 12.209278106689453
Epoch 0 MSE DIR 21.485130310058594 MSE VAR 0.4950293004512787 KLD 17.081342697143555
Epoch 0 MSE DIR 19.008243560791016 MSE VAR 0.49411502480506897 KLD 23.224525451660156
Epoch 0 MSE DIR 61.19855499267578 MSE VAR 0.4858565330505371 KLD 21.71772003173828
Epoch 0 MSE DIR 62.53136444091797 MSE VAR 0.4833485186100006 KLD 17.621334075927734
Epoch 0 MSE DIR 49.16545486450195 MSE VAR 0.48443081974983215 KLD 19.106746673583984
Epoch 0 MSE DIR 19.143146514892578 MSE VAR 0.4886527955532074 KLD 17.897201538085938
Epoch 0 MSE DIR 32.7155876159668 MSE VAR 0.4929197132587433 KLD 18.229

Epoch 0 MSE DIR 29.75576400756836 MSE VAR 0.49339383840560913 KLD 19.597007751464844
Epoch 0 MSE DIR 20.450279235839844 MSE VAR 0.4863625466823578 KLD 24.412830352783203
Epoch 0 MSE DIR 232.92926025390625 MSE VAR 0.4851354658603668 KLD 14.903364181518555
Epoch 0 MSE DIR 48.88629913330078 MSE VAR 0.4806923270225525 KLD 18.06434440612793
Epoch 0 MSE DIR 19.654464721679688 MSE VAR 0.486275315284729 KLD 20.141889572143555
Epoch 0 MSE DIR 54.6407585144043 MSE VAR 0.4909486770629883 KLD 16.580142974853516
Epoch 0 MSE DIR 74.94363403320312 MSE VAR 0.4926506280899048 KLD 20.770584106445312
Epoch 0 MSE DIR 32.430274963378906 MSE VAR 0.49357840418815613 KLD 21.188575744628906
Epoch 0 MSE DIR 92.7341079711914 MSE VAR 0.4887000322341919 KLD 21.025354385375977
Epoch 0 MSE DIR 117.94430541992188 MSE VAR 0.47942572832107544 KLD 20.7138729095459
Epoch 0 MSE DIR 11.015804290771484 MSE VAR 0.4799269437789917 KLD 26.985862731933594
Epoch 0 MSE DIR 49.39066696166992 MSE VAR 0.4909909963607788 KLD 23.96019

Epoch 0 MSE DIR 65.18696594238281 MSE VAR 0.4936406910419464 KLD 16.93975257873535
Epoch 0 MSE DIR 27.360654830932617 MSE VAR 0.49033406376838684 KLD 20.579952239990234
Epoch 0 MSE DIR 57.257835388183594 MSE VAR 0.4809683561325073 KLD 14.748924255371094
Epoch 0 MSE DIR 43.73590850830078 MSE VAR 0.48042747378349304 KLD 21.85757827758789
Epoch 0 MSE DIR 36.90272903442383 MSE VAR 0.49207642674446106 KLD 23.14450454711914
Epoch 0 MSE DIR 135.73257446289062 MSE VAR 0.4971372187137604 KLD 16.044801712036133
Epoch 0 MSE DIR 51.17812728881836 MSE VAR 0.48745524883270264 KLD 19.719833374023438
Epoch 0 MSE DIR 27.910743713378906 MSE VAR 0.48248666524887085 KLD 23.181262969970703
Epoch 0 MSE DIR 46.4608268737793 MSE VAR 0.481234610080719 KLD 14.227428436279297
Epoch 0 MSE DIR 192.34677124023438 MSE VAR 0.4898410439491272 KLD 14.70449161529541
Epoch 0 MSE DIR 66.88955688476562 MSE VAR 0.494772732257843 KLD 16.112510681152344
Epoch 0 MSE DIR 72.49478912353516 MSE VAR 0.4937962293624878 KLD 17.20683

Epoch 0 MSE DIR 38.53894805908203 MSE VAR 0.48461079597473145 KLD 21.424068450927734
Epoch 0 MSE DIR 120.11686706542969 MSE VAR 0.48972365260124207 KLD 21.393810272216797
Epoch 0 MSE DIR 20.009326934814453 MSE VAR 0.49449658393859863 KLD 21.000919342041016
Epoch 0 MSE DIR 49.76842498779297 MSE VAR 0.49235665798187256 KLD 19.147823333740234
Epoch 0 MSE DIR 168.87515258789062 MSE VAR 0.4808998703956604 KLD 18.952083587646484
Epoch 0 MSE DIR 107.23477172851562 MSE VAR 0.48107486963272095 KLD 19.7667179107666
Epoch 0 MSE DIR 270.22900390625 MSE VAR 0.47655731439590454 KLD 16.409423828125
Epoch 0 MSE DIR 98.76374816894531 MSE VAR 0.5013583898544312 KLD 17.581165313720703
Epoch 0 MSE DIR 341.1606750488281 MSE VAR 0.5030604004859924 KLD 21.215587615966797
Epoch 0 MSE DIR 154.86746215820312 MSE VAR 0.48710501194000244 KLD 13.861568450927734
Epoch 0 MSE DIR 114.57585144042969 MSE VAR 0.4775637984275818 KLD 13.942689895629883
Epoch 0 MSE DIR 35.11960220336914 MSE VAR 0.47776055335998535 KLD 19.2

KeyboardInterrupt: 

In [ ]:
res_eval_2, res_vae_2, test_list_2 = agent.train(training_iter=50000,
                                           env_name=env_name,
                                           seed=0,
                                           task_generator=task_generator,
                                           eval_interval=100,
                                           log_dir=".",
                                           use_env_obs=False,
                                           num_vae_steps=1,
                                           init_vae_steps=1,
                                           sw_size=10,
                                           num_random_task_to_eval=32,
                                           num_test_processes=2,
                                           use_data_loader=False,
                                           gp_list_sequences=[],
                                           prior_sequences=[],
                                           init_prior_test_sequences=[],
                                           verbose=True
                                          )

In [ ]:
vi_2 = InferenceNetwork(n_in=4, z_dim=latent_dim)
vi_optim_2 = torch.optim.Adam(vi.parameters(), lr=1e-3)

agent_2 = PosteriorMTAgent(action_space=action_space, device=device, gamma=1,
                                 num_steps=20, num_processes=32,
                                 clip_param=0.1, ppo_epoch=4,
                                 num_mini_batch=8,
                                 value_loss_coef=0.5,
                                 entropy_coef=0.001,
                                 lr=0.000005,
                                 eps=1e-6, max_grad_norm=0.5,
                                 use_linear_lr_decay=False,
                                 use_gae=False,
                                 gae_lambda=0.95,
                                 use_proper_time_limits=False,
                                 obs_shape=obs_shape,
                                 latent_dim=latent_dim,
                                 recurrent_policy=False,
                                 hidden_size=8,
                                 use_elu=True,
                                 variational_model=vi_2,
                                 vae_optim=vi_optim_2,
                                 rescale_obs=True,
                                 max_old=max_old,
                                 min_old=min_old,
                                 vae_min_seq=vae_min_seq,
                                 vae_max_seq=vae_max_seq,
                                 max_action=x_max, 
                                 min_action=x_min,
                        use_time=False, rescale_time=None, max_time=None,
                        max_sigma=30,
                        use_decay_kld=True,
                        decay_kld_rate=1)

res_eval_2, res_vae_2, test_list_2 = agent_2.train(training_iter=60000,
                                           env_name=env_name,
                                           seed=0,
                                           task_generator=task_generator,
                                           eval_interval=100,
                                           log_dir=".",
                                           use_env_obs=False,
                                           init_vae_steps=1,
                                           sw_size=10,
                                           num_random_task_to_eval=32,
                                           num_test_processes=2,
                                           use_data_loader=False,
                                           gp_list_sequences=[],
                                           prior_sequences=[],
                                           init_prior_test_sequences=[],
                                           verbose=True
                                          )